# Make the A-list

The A-list is the list of objects with all final products XID+ fluxes, redshift, and CIGALE physical properties.

Here we will loop over all the final catalogues, take the xid+ objects, and the set of columns present on every field. We will also add new 'best' photometry where for every common band type we take the valeu with the lowest error.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
017bb1e (Mon Jun 18 14:58:59 2018 +0100) [with local modifications]
This notebook was executed on: 
2020-11-05 17:25:29.916077


In [2]:
from astropy.table import Table, vstack
import glob
import yaml
import os
import time
import gc
import numpy as np
from herschelhelp_internal.utils import add_column_meta
SUFFIX = os.environ.get('SUFFIX', time.strftime("%Y%m%d"))
print(SUFFIX)

MAKE_FULL = True

20201105


In [3]:
bands = ['u', 'g', 'r', 'i', 'z', 'y', 'j', 'h', 'k', 'ks']
always_keep = ['irac','mips','pacs','spire', 'best']

In [4]:
meta_files = glob.glob('../*/meta_main.yml')

In [5]:
meta_files

['../dmu32_SA13/meta_main.yml',
 '../dmu32_Bootes/meta_main.yml',
 '../dmu32_xFLS/meta_main.yml',
 '../dmu32_GAMA-15/meta_main.yml',
 '../dmu32_GAMA-12/meta_main.yml',
 '../dmu32_HDF-N/meta_main.yml',
 '../dmu32_NGP/meta_main.yml',
 '../dmu32_ELAIS-N2/meta_main.yml',
 '../dmu32_EGS/meta_main.yml',
 '../dmu32_COSMOS/meta_main.yml',
 '../dmu32_SGP/meta_main.yml',
 '../dmu32_Lockman-SWIRE/meta_main.yml',
 '../dmu32_XMM-LSS/meta_main.yml',
 '../dmu32_ELAIS-N1/meta_main.yml',
 '../dmu32_AKARI-SEP/meta_main.yml',
 '../dmu32_GAMA-09/meta_main.yml',
 '../dmu32_AKARI-NEP/meta_main.yml',
 '../dmu32_ELAIS-S1/meta_main.yml',
 '../dmu32_XMM-13hr/meta_main.yml',
 '../dmu32_Herschel-Stripe-82/meta_main.yml',
 '../dmu32_CDFS-SWIRE/meta_main.yml',
 '../dmu32_SPIRE-NEP/meta_main.yml',
 '../dmu32_SSDF/meta_main.yml']

In [6]:
def get_best_flux(table, band):
    "Return the lowest error non nan band "
    #table=table.copy()
    f_best = np.full(len(table), np.nan)
    ferr_best = np.full(len(table), np.nan)
    m_best= np.full(len(table), np.nan)
    merr_best= np.full(len(table), np.nan)
    flag_best= np.full(len(table), False)
    camera = np.full(len(table), 'band_not_available')
    
    for col in table.colnames:
        if (col.startswith('m_') 
            and (not col.startswith('m_ap'))
            and col.endswith('_{}'.format(band))):
            #replace all nans
            m1 = np.isnan(f_best)
            f_best[m1] = table[m1]['f' + col[1:]]
            ferr_best[m1] = table[m1]['ferr' + col[1:]]
            m_best[m1] = table[m1][col]
            merr_best[m1] = table[m1]['merr' + col[1:]]
            try:
                flag_best[m1] = table[m1]['flag' + col[1:]]
            except:
                'NO FLAG COL FOR {}'.format(col)
            m1= m1 & (~np.isnan(table[col]))
            camera[m1] = np.full(np.sum(m1),col[2:])
            #replace all with larger flux error
            m2 = table['ferr' + col[1:]] < ferr_best
            f_best[m2] = table[m2]['f' + col[1:]]
            ferr_best[m2] = table[m2]['ferr' + col[1:]]
            m_best[m2] = table[m2][col]
            merr_best[m2] = table[m2]['merr' + col[1:]]
            try:
                flag_best[m2] = table[m2]['flag' + col[1:]]
            except:
                'NO FLAG COL FOR {}'.format(col)
            camera[m2] = np.full(np.sum(m2),col.split('_')[-2])
    
    return f_best, ferr_best, m_best, merr_best, flag_best, camera
    
def remove_specific_bands(table, bands_to_keep):
    "Return the table with all specific band fluxes removed"
    #table = table.copy()
    cols_to_remove = set()
    for col in table.colnames:
        if col.startswith('m_ap_'):
            to_remove=(
                col,  
                'merr_ap' + col[4:],
                'f_ap' + col[4:],
                'ferr_ap' + col[4:]
            )
            cols_to_remove = cols_to_remove.union(set(to_remove))
    
    
    
    for col in table.colnames:
        if col.startswith('m_') and (col.split('_')[-2] not in bands_to_keep):
            #print(col.split('_')[-2])
            to_remove =(
                col,  
                'merr' + col[1:],
                'f' + col[1:],
                'ferr' + col[1:],
                'flag' + col[1:]
                )
            cols_to_remove = cols_to_remove.union(set(to_remove))
            
    #print('Removing ',cols_to_remove)
    for c in cols_to_remove:
        try:
            table.remove_column(c)
        except:
            continue
            #print('Column {} cannot be removed it is not there'.format(c))
            
            
            
    return table

In [7]:
full_cat = Table()
full_sources = Table()
for meta_file in meta_files: #meta_files[16:17]:
    meta = yaml.safe_load(open(meta_file, 'r'))
    final_cat_loc = meta['final'].replace('dmu_products', '../..')
    #final_cat_loc = final_cat_loc.replace('_cigale','')
    #print(final_cat_loc)
    try:
        final_cat = Table.read(final_cat_loc)
        print(final_cat_loc)
    except:
        continue
        
    if not MAKE_FULL:
        final_cat = final_cat[final_cat['f_spire_500']>0]
        
    #remove duplicates - this problem will be fixed upstream in dmu2 merging but this is a quick hack
    final_cat.sort('help_id')
    is_duplicate = final_cat['help_id'][1:] == final_cat['help_id'][:-1]
    if np.sum(is_duplicate)>0:
        print('Field {} has duplicates... removing them!'.format(meta['field']))
    final_cat = final_cat[~np.append(is_duplicate,False)]
    
    sources = Table()
    sources['help_id'] = final_cat['help_id']
    
    for band in bands:
        f_best, ferr_best, m_best, merr_best, flag_best, camera = get_best_flux(final_cat, band)
        final_cat['f_best_{}'.format(band)] = f_best
        final_cat['ferr_best_{}'.format(band)] = ferr_best
        final_cat['m_best_{}'.format(band)] = m_best
        final_cat['merr_best_{}'.format(band)] = merr_best
        final_cat['flag_best_{}'.format(band)] = flag_best
        sources['best_{}'.format(band)] = camera
    
    
    final_cat = remove_specific_bands(final_cat,always_keep)
    
    if MAKE_FULL:
        m=final_cat['f_spire_500'] > 0.
        final_cat.write('./data/{}_best_phot_{}.fits'.format(meta['field'],SUFFIX), overwrite=True)
        final_cat = final_cat[m]
        sources.write('./data/{}_best_phot_sources_{}.fits'.format(meta['field'],SUFFIX), overwrite=True) 
        sources= sources[m]
    
    full_cat = vstack([full_cat,final_cat])                                           
    full_sources = vstack([full_sources,sources])
    
 
    del final_cat
    del sources
    gc.collect()
    time.sleep(10)

../../dmu32/dmu32_SA13/data/SA13_20180501.fits


/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)
/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)


../../dmu32/dmu32_Bootes/data/Bootes_20190701.fits


/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)
/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)


../../dmu32/dmu32_xFLS/data/xFLS_20180501.fits


...

/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)
/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)


../../dmu32/dmu32_SPIRE-NEP/data/SPIRE-NEP_20180220.fits


/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)
/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)


../../dmu32/dmu32_SSDF/data/SSDF_20180221.fits


/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)
/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)


In [8]:
full_cat = full_cat.filled()
full_cat = add_column_meta(full_cat, 'columns.yml')

Column old_help_id has missing data


In [9]:
full_cat[-5:]

help_id,field,ra,dec,hp_idx,ebv,redshift,zspec,f_spire_250,ferr_spire_250,flag_spire_250,f_spire_350,ferr_spire_350,flag_spire_350,f_spire_500,ferr_spire_500,flag_spire_500,cigale_mstar,cigale_mstar_err,cigale_sfr,cigale_sfr_err,cigale_dustlumin,cigale_dustlumin_err,cigale_dustlumin_ironly,cigale_dustlumin_ironly_err,cigale_chi2_opt,cigale_chi2_ir,cigale_chi2_red,cigale_chi2_ironly,stellarity,stellarity_origin,flag_cleaned,flag_merged,flag_gaia,flag_optnir_obs,flag_optnir_det,zspec_qual,zspec_association_flag,f_best_u,ferr_best_u,m_best_u,merr_best_u,flag_best_u,f_best_g,ferr_best_g,m_best_g,merr_best_g,flag_best_g,f_best_r,ferr_best_r,m_best_r,merr_best_r,flag_best_r,f_best_i,ferr_best_i,m_best_i,merr_best_i,flag_best_i,f_best_z,ferr_best_z,m_best_z,merr_best_z,flag_best_z,f_best_y,ferr_best_y,m_best_y,merr_best_y,flag_best_y,f_best_j,ferr_best_j,m_best_j,merr_best_j,flag_best_j,f_best_h,ferr_best_h,m_best_h,merr_best_h,flag_best_h,f_best_k,ferr_best_k,m_best_k,merr_best_k,flag_best_k,f_best_ks,ferr_best_ks,m_best_ks,merr_best_ks,flag_best_ks,f_irac_i1,ferr_irac_i1,m_irac_i1,merr_irac_i1,f_irac_i2,ferr_irac_i2,m_irac_i2,merr_irac_i2,f_irac_i3,ferr_irac_i3,m_irac_i3,merr_irac_i3,f_irac_i4,ferr_irac_i4,m_irac_i4,merr_irac_i4,f_mips_24,ferr_mips_24,flag_mips_24,f_pacs_green,ferr_pacs_green,flag_pacs_green,f_pacs_red,ferr_pacs_red,flag_pacs_red,flag_irac_i1,flag_irac_i2,flag_irac_i3,flag_irac_i4,old_help_id
,,deg,deg,,,,,uJy,uJy,,uJy,uJy,,uJy,uJy,,solMass,solMass,solMass / yr,solMass / yr,W,W,W,W,,,,,,,,,,,,,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,uJy,uJy,mag,mag,uJy,uJy,mag,mag,uJy,uJy,mag,mag,uJy,uJy,,uJy,uJy,,uJy,uJy,,,,,,
bytes33,bytes18,float64,float64,int64,float64,float64,float64,float64,float64,bool,float64,float64,bool,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes20,bool,bool,int64,int64,int64,int64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,bool,float64,float64,bool,bool,bool,bool,bool,bytes27
HELP_J235959.991-572600.759,SSDF,359.99996065287047,-57.4335442131269,756110660,0.011629759520262972,0.1883,nan,2415.7939453125,3492.63330078125,True,4109.18359375,4760.20703125,True,5863.16796875,7108.9755859375,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.06470298767089844,des_stellarity,False,False,0,7,5,-99,False,nan,nan,nan,nan,False,32.99696290274807,0.4018202315537077,20.10381507873535,0.013221543282270432,False,114.09133319331883,0.5833148505822463,18.756868362426758,0.00555104436352849,False,171.737731773923,0.9955295632196581,18.312835693359375,0.0062937973998487,False,219.0555188603023,1.8208271095337143,18.048614501953125,0.009024825878441334,False,249.55869099850693,5.9419066181059055,17.907068252563477,0.02585100568830967,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,291.4200134277344,8.739999771118164,17.73870086669922,0.03256239369511604,213.88999938964844,6.420000076293945,18.07452392578125,0.032588835805654526,1e+20,1e+20,1e+20,1e+20,1e+20,1e+20,1e+20,1e+20,1e+20,1e+20,True,1e+20,1e+20,True,1e+20,1e+20,True,False,False,True,True,N/A
HELP_J235959.993-571303.407,SSDF,359.99997079239654,-57.217613000720895,756285780,0.013104888166929295,0.916,nan,3221.186279296875,3619.668701171875,True,7596.54736328125,6319.90380859375,True,3466.42529296875,53

In [10]:
len(full_cat) == len(np.unique(full_cat['help_id']))

True

In [11]:
len(full_cat.colnames)

118

In [12]:
full_cat.colnames

['help_id',
 'field',
 'ra',
 'dec',
 'hp_idx',
 'ebv',
 'redshift',
 'zspec',
 'f_spire_250',
 'ferr_spire_250',
 'flag_spire_250',
 'f_spire_350',
 'ferr_spire_350',
 'flag_spire_350',
 'f_spire_500',
 'ferr_spire_500',
 'flag_spire_500',
 'cigale_mstar',
 'cigale_mstar_err',
 'cigale_sfr',
 'cigale_sfr_err',
 'cigale_dustlumin',
 'cigale_dustlumin_err',
 'cigale_dustlumin_ironly',
 'cigale_dustlumin_ironly_err',
 'cigale_chi2_opt',
 'cigale_chi2_ir',
 'cigale_chi2_red',
 'cigale_chi2_ironly',
 'stellarity',
 'stellarity_origin',
 'flag_cleaned',
 'flag_merged',
 'flag_gaia',
 'flag_optnir_obs',
 'flag_optnir_det',
 'zspec_qual',
 'zspec_association_flag',
 'f_best_u',
 'ferr_best_u',
 'm_best_u',
 'merr_best_u',
 'flag_best_u',
 'f_best_g',
 'ferr_best_g',
 'm_best_g',
 'merr_best_g',
 'flag_best_g',
 'f_best_r',
 'ferr_best_r',
 'm_best_r',
 'merr_best_r',
 'flag_best_r',
 'f_best_i',
 'ferr_best_i',
 'm_best_i',
 'merr_best_i',
 'flag_best_i',
 'f_best_z',
 'ferr_best_z',
 'm_best

In [13]:
full_sources[:5]

help_id,best_u,best_g,best_r,best_i,best_z,best_y,best_j,best_h,best_k,best_ks
bytes33,str18,str18,str18,str18,str18,str18,str18,str18,str18,str18
HELP_J131038.491+424429.719,band_not_available,90prime_g,90prime_r,band_not_available,mosaic_z,band_not_available,ukidss_j,band_not_available,band_not_available,band_not_available
HELP_J131040.639+424529.290,band_not_available,90prime_g,90prime_r,band_not_available,mosaic_z,band_not_available,ukidss_j,band_not_available,band_not_available,band_not_available
HELP_J131041.208+425230.397,band_not_available,90prime_g,90prime_r,band_not_available,mosaic_z,band_not_available,ukidss_j,band_not_available,band_not_available,band_not_available
HELP_J131041.287+424511.374,band_not_available,90prime_g,90prime_r,band_not_available,mosaic_z,band_not_available,ukidss_j,band_not_available,band_not_available,band_not_available
HELP_J131041.685+424358.772,band_not_available,90prime_g,90prime_r,band_not_available,mosaic_z,band_not_available,ukidss_j,band_not_available,band_not_available,band_not_available


In [14]:
!mkdir data
full_cat.write('./data/HELP_all_sky_xid_prior_{}.fits'.format(SUFFIX), overwrite=True)
full_sources.write('./data/HELP_all_sky_xid_prior_sources_{}.fits'.format(SUFFIX), overwrite=True)
#full_sources.write('./data/HELP_all_sky_xid_objects_sources_{}.csv'.format(SUFFIX), overwrite=True)

mkdir: cannot create directory ‘data’: File exists


In [15]:
#just cigale objects to make a smaller file
m = full_cat['cigale_sfr']>0.
full_cat = full_cat[m]
full_sources = full_sources[m]
print(len(full_cat))
full_cat.write('./data/HELP_all_sky_A-list_{}.fits'.format(SUFFIX), overwrite=True)
full_sources.write('./data/HELP_all_sky_A-list_sources_{}.fits'.format(SUFFIX), overwrite=True)
#full_cat.write('./data/HELP_all_sky_xid_A-list_{}.csv'.format(SUFFIX), overwrite=True)
#full_sources.write('./data/HELP_all_sky_xid_objects_sources_{}.fits'.format(SUFFIX), overwrite=True)
#full_sources.write('./data/HELP_all_sky_xid_objects_sources_{}.csv'.format(SUFFIX), overwrite=True)

/opt/pyenv/versions/3.7.2/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)


1718100
